In [1]:
from common import *

In [2]:
import requests
from tqdm import tqdm
import copy

# Notion API 키 및 데이터베이스 ID 설정
notion = readpkl('notion.pkl')
NOTION_API_KEY = notion['NOTION_API_KEY']
DATABASE_ID = notion['DATABASE_ID']

# 헤더 설정
headers = {
    "Authorization": f"Bearer {NOTION_API_KEY}",
    "Content-Type": "application/json",
    "Notion-Version": "2022-06-28"
}

### Check experiment page existence

In [11]:
pages_dict, model_dict = get_notion_dicts(NOTION_API_KEY, DATABASE_ID, headers)

# check to-make list
to_make_parent = []
to_make_child = {}
for experiment in experiment_list:
    child_list = []
    with open(f'experiments/{experiment}.txt', 'r') as f:
        lines = f.readlines()
        for line in lines:
            child_list.append(line.split('\n')[0].strip())
    num_childs = len(child_list)
    if experiment not in model_dict.keys():
        to_make_parent.append(experiment)
        to_make_child[experiment] = child_list
    else:
        model_ids = model_dict[experiment]
        if len(model_ids) > 1: # if there is a child
            # check parent id
            for model_id in model_ids:
                is_parent = pages_dict[model_id]['properties']['Is_parent']['checkbox']
                if is_parent: 
                    parent_id = model_id
                    break
            child_ids = [model_id for model_id in model_ids if model_id != parent_id]
            child_page_list = [pages_dict[child_id]['properties']['Name']['title'][0]['text']['content'] for child_id in child_ids]
            if len(child_page_list) < num_childs: to_make_child[experiment] = list(set(child_list) - set(child_page_list))
        else:
            parent_id = model_ids[0]
            child_ids = []
            child_page_list = []
            to_make_child[experiment] = child_list
            
to_make_parent, to_make_child

Total pages: 309


([], {})

In [4]:
# Create parent pages
for checkpoint in to_make_parent:
    # create new page
    if 'MB' in checkpoint: architecture = 'MotionBERT'
    else: architecture = None
    
    if 'frame_27' in checkpoint: input_frame = 27
    elif 'frame_81' in checkpoint: input_frame = 81
    else: input_frame = 243
    
    if 'norm_input_scale' in checkpoint: input_scale = 'Scaled by Euclidean norm'
    else: input_scale = 'X'
    
    train_dataset, test_dataset = select_dataset_from_checkpoint(checkpoint)
    
    parent_properties = {
        "Name": {"title": [{'type': 'text', 'text': {'content': checkpoint}}]},
        #"Model": {"select": {"name": checkpoint}},
        "Is_parent": {"checkbox": True},
        "Architecture": {"select": {"name": architecture}},
        "Input frame": {"number": input_frame},
        "Train dataset": {"select": {"name": train_dataset}},
        "Test dataset": {"select": {"name": test_dataset}},
        "Input scaling": {"select": {"name": input_scale}},
    }
    res = create_notion_page(DATABASE_ID, headers, parent_properties, children=None)
    print(res)
    
# Reload notion pages
pages_dict, model_dict = get_notion_dicts(NOTION_API_KEY, DATABASE_ID, headers)

Total pages: 289


In [5]:
# Reload notion pages
#pages_dict, model_dict = get_notion_dicts(NOTION_API_KEY, DATABASE_ID, headers)

# Create child pages
for checkpoint in to_make_child.keys():
    model_ids = model_dict[checkpoint]
    parent_id = None
    for model_id in model_ids:
        is_parent = pages_dict[model_id]['properties']['Is_parent']['checkbox']
        if is_parent: 
            parent_id = model_id
            break
    if parent_id is None: 
        print(f'Parent page not found for {checkpoint}')
        break
    parent_properties = pages_dict[parent_id]['properties']
    model = parent_properties['Model']
    architecture = parent_properties['Architecture']
    input_frame = parent_properties['Input frame']
    input_scaling = parent_properties['Input scaling']
    canonical = parent_properties['Canonical']
    train_dataset = parent_properties['Train dataset']
    
    child_list = to_make_child[checkpoint]
    for subset in child_list:
        test_dataset = select_testset_from_subset(subset)
        child_properties = {
            "Name": {"title": [{"text": {"content": subset}}]},
            "Is_parent": {"checkbox": False},
            "Architecture": architecture,
            "Canonical": canonical,
            "Input frame": input_frame,
            "Input scaling": input_scaling,
            "Train dataset": train_dataset,
            "Test dataset": {"select": {"name": test_dataset}},
            "Parent item": {'id': 'FQBZ', 'type': 'relation', 'relation': [{'id': parent_id}], 'has_more': False}
        }
        res = create_notion_page(DATABASE_ID, headers, child_properties, children=None)
        print(res)

{'object': 'page', 'id': '35113a60-0b1a-4f7f-8175-82e384c4ad0a', 'created_time': '2024-08-14T01:34:00.000Z', 'last_edited_time': '2024-08-14T01:34:00.000Z', 'created_by': {'object': 'user', 'id': 'c6d38448-0013-4c0a-91c8-359f6979bf12'}, 'last_edited_by': {'object': 'user', 'id': 'c6d38448-0013-4c0a-91c8-359f6979bf12'}, 'cover': None, 'icon': None, 'parent': {'type': 'database_id', 'database_id': '697fd4ef-9dca-42f7-a204-bd1b57e64b83'}, 'archived': False, 'in_trash': False, 'properties': {'Input frame': {'id': '%3BvPG', 'type': 'number', 'number': 243}, 'Train 2D input': {'id': '%3FY%7BB', 'type': 'formula', 'formula': {'type': 'string', 'string': 'SH'}}, 'P-MPJPE_prev': {'id': 'Avig', 'type': 'number', 'number': None}, 'Parent item': {'id': 'FQBZ', 'type': 'relation', 'relation': [{'id': '021a4a61-24fb-47e7-a6d2-902f6e17808f'}], 'has_more': False}, 'Process': {'id': 'FiF%5E', 'type': 'select', 'select': None}, 'MPJPE_prev': {'id': 'KC%3C%7D', 'type': 'number', 'number': None}, 'Input s

In [8]:
# read result_dict
result_dict = readpkl('result_dict.pkl')

### Compare before and after 

In [4]:
subset_dict = {}
for checkpoint in result_dict.keys():
    if 'cam_no_factor' not in checkpoint: continue
    print(f'\n{checkpoint}')
    model_ids = model_dict[checkpoint]
    for model_id in model_ids:
        page = pages_dict[model_id]
        train = page['properties']['Train dataset']['select'] 
        test = page['properties']['Test dataset']['select']
        # filtering - parent
        if test == None: continue # parent
        
        # get child page's info
        child_page = pages_dict[model_id]
        child_properties = child_page['properties']
        if len(child_properties['Name']['title']) == 0: continue
        child_title = child_properties['Name']['title'][0]['text']['content']
        if child_title in result_dict[checkpoint].keys():
            mpjpe = child_properties['MPJPE_prev']['number']
            p_mpjpe = child_properties['P-MPJPE_prev']['number']
            if mpjpe == None: mpjpe = -1
            if p_mpjpe == None: p_mpjpe = -1
            if len(result_dict[checkpoint][child_title].keys()) >0 :
                #print(f'\t{child_title}')
                e1 = result_dict[checkpoint][child_title]["e1"]
                e2 = result_dict[checkpoint][child_title]["e2"]
                if e1 == None: e1 = -1
                if e2 == None: e2 = -1
            else:
                e1 = -1
                e2 = -1
            if e1 > 0 and mpjpe > 0:
                if e1 - mpjpe > 1: mark = '⬆️'
                elif e1 - mpjpe < -1: mark = '⬇️'
                else: mark = '✅'
            else:
                mark = '❌'
            print(f'{mark}\t{e1:.2f} {e2:.2f} <- {mpjpe:.2f} {p_mpjpe:.2f}\t{child_title}')
            if child_title not in subset_dict: subset_dict[child_title] = []
            subset_dict[child_title].append(mark)


MB_train_fit3d_gt_cam_no_factor_input_from_canonical_3d_same_z_ts_s4710
✅	159.42 117.13 <- 159.63 117.19	3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_Z-TEST_ALL_TRAIN
⬇️	167.25 121.53 <- 288.42 154.17	3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_Z-TEST_TS1_6
✅	113.02 84.69 <- 113.02 84.69	H36M-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_Z
✅	108.62 79.20 <- 108.62 79.20	H36M-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_Z-TR_S1_TS_S5678
✅	30.49 14.50 <- 30.49 14.50	FIT3D-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_Z-TS_S4710

MB_train_fit3d_gt_cam_no_factor_with_canonical2_tr_s03
❌	186.13 125.26 <- -1.00 -1.00	3DHP-GT-CAM_NO_FACTOR-TEST_ALL_TRAIN
❌	177.16 118.93 <- -1.00 -1.00	3DHP-GT-CAM_NO_FACTOR-TEST_TS1_6
❌	41.92 26.26 <- -1.00 -1.00	FIT3D-GT-CAM_NO_FACTOR-TR_S03
❌	130.98 94.77 <- -1.00 -1.00	H36M-GT-CAM_NO_FACTOR-TR_S1_TS_S5678
❌	133.72 100.37 <- -1.00 -1.00	H36M-GT-CAM_NO_FACTOR

MB_train_fit3d_gt_cam_no_factor_input_from_canonical_3d_same_z_input_centering_tr_s03

KeyError: 'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_fixed_dist_5_input_centering_tr_s1_ts_s5678_frame_27'

In [9]:
for subset in natsorted(subset_dict):
    print(subset, subset_dict[subset])

3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_FIXED_DIST_5-TEST_ALL_TRAIN ['❌', '❌']
3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_FIXED_DIST_5-TEST_TS1_6 ['❌', '❌']
3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_Z-TEST_ALL_TRAIN ['✅', '⬆️', '⬇️', '✅', '⬆️', '⬇️', '⬇️', '⬇️']
3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_Z-TEST_TS1_6 ['⬇️', '⬆️', '⬆️', '⬆️', '⬇️', '⬇️', '⬇️', '⬇️']
3DHP-GT-CAM_NO_FACTOR-TEST_ALL_TRAIN ['⬇️', '⬇️', '⬇️', '⬇️', '✅', '⬇️', '⬇️', '⬇️']
3DHP-GT-CAM_NO_FACTOR-TEST_TS1_6 ['⬇️', '⬇️', '⬆️', '⬇️', '⬇️', '⬇️', '⬇️', '⬆️']
FIT3D-GT-CAM_NO_FACTOR-ALL_TEST ['⬇️', '⬇️', '⬇️', '⬇️', '⬇️']
FIT3D-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_FIXED_DIST_5-ALL_TEST ['✅', '✅']
FIT3D-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_Z-ALL_TEST ['✅', '⬇️', '✅', '✅', '✅']
FIT3D-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_Z-TR_S03 ['⬇️', '⬇️']
FIT3D-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_Z-TS_S4710 ['✅', '✅']
FIT3D-GT-CAM_NO_FACTOR-TR_S03 ['⬇️']
FIT3D-GT-CAM_NO_FACTO

### Update notion pages

In [12]:
for checkpoint in result_dict.keys():
    if 'cam_no_factor' not in checkpoint: continue
    print(f'\n{checkpoint}')
    model_ids = model_dict[checkpoint]
    if len(model_ids) > 1: # if there is a child
        # check parent id
        for model_id in model_ids:
            page = pages_dict[model_id]
            is_parent = page['properties']['Is_parent']['checkbox']
            if is_parent: 
                parent_id = model_id
                break
        
        for model_id in model_ids:  
            if model_id == parent_id: continue # filtering - parent
            # get child page's info
            child_page = pages_dict[model_id]
            child_properties = child_page['properties']
            if len(child_properties['Name']['title']) == 0: continue
            child_title = child_properties['Name']['title'][0]['text']['content']
            child_mpjpe = child_properties['MPJPE']['number']
            child_p_mpjpe = child_properties['P-MPJPE']['number']
            if child_mpjpe != None and child_p_mpjpe != None: continue
            
            if child_title in result_dict[checkpoint].keys():
                e1 = result_dict[checkpoint][child_title]["e1"]
                e2 = result_dict[checkpoint][child_title]["e2"]
                if e1 == None: e1 = -1
                if e2 == None: e2 = -1
                update_mpjpe(model_id, headers, float(f'{e1:.2f}'), float(f'{e2:.2f}'))
            


MB_train_fit3d_gt_cam_no_factor_input_from_canonical_3d_same_z_ts_s4710

MB_train_fit3d_gt_cam_no_factor_with_canonical2_tr_s03

MB_train_fit3d_gt_cam_no_factor_input_from_canonical_3d_same_z_input_centering_tr_s03

MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_same_z_input_centering_tr_s1_ts_s5678

MB_train_fit3d_gt_cam_no_factor_input_from_canonical_3d_same_z_input_centering_ts_s4710

MB_train_h36m_gt_cam_no_factor

MB_train_fit3d_gt_cam_no_factor_ts_s4710

MB_train_h36m_gt_cam_no_factor_with_canonical2_tr_s1_ts_s5678

MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_fixed_dist_5_input_centering

MB_train_h36m_gt_cam_no_factor_tr_s1_ts_s5678

MB_train_fit3d_gt_cam_no_factor_input_from_canonical_3d_same_z_tr_s03

MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_fixed_dist_5_input_centering_tr_s1_ts_s5678

MB_train_h36m_gt_cam_no_factor_s15678_tr_54138969_ts_others

MB_train_fit3d_gt_cam_no_factor_tr_s03

MB_train_h36m_gt_cam_no_factor_with_canonical2

MB_train_h3

### Check test dataset

In [14]:
# Reload notion pages
pages_dict, model_dict = get_notion_dicts(NOTION_API_KEY, DATABASE_ID, headers)

Total pages: 279


In [15]:
for experiment in experiment_list:
    model_ids = model_dict[experiment]
    for model_id in model_ids:
        page = pages_dict[model_id]
        try:
            test = page['properties']['Test dataset']['select']['name']
        except:
            print(f'{experiment} has no test dataset')

MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_same_z has no test dataset
MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_same_z_input_centering_tr_s1_ts_s5678 has no test dataset
MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_fixed_dist_5_adaptive_focal_input_centering_tr_s1_ts_s5678 has no test dataset
MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_fixed_dist_5_input_centering_tr_s1_ts_s5678 has no test dataset
MB_train_fit3d_gt_cam_no_factor_input_from_canonical_3d_same_z_input_centering_ts_s4710 has no test dataset
MB_train_fit3d_gt_cam_no_factor_input_from_canonical_3d_same_z_input_centering_ts_s4710 has no test dataset
MB_train_fit3d_gt_cam_no_factor_with_canonical2_ts_s4710 has no test dataset
MB_train_h36m_gt_cam_no_factor_with_canonical2_tr_s1 has no test dataset
MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_same_z_s15678_tr_54138969_ts_others has no test dataset
MB_train_h36m_gt_cam_no_factor_with_canonical2_tr_s1_ts_s5678 has no test d